In [1]:
import pandas as pd
import numpy as np
import random
from sympy import isprime, nextprime
import os
import pickle
os.chdir('Resources/')

In [2]:
def load_key(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [3]:
def string_to_int(message):
    return int.from_bytes(message.encode('utf-8'), byteorder='big')

def int_to_string(value):
    byte_length = (value.bit_length() + 7) // 8 
    return value.to_bytes(byte_length, byteorder='big').decode('utf-8', errors='ignore')

def float_to_string(message):
    return str(message)

def string_to_float(message):
    return float(message)

In [4]:
def power(b, p, m):
    b = b % m
    if p == 0:
        return 1
    j = power(b, p // 2, m)
    j = (j * j) % m
    if p % 2 == 1:
        j = (j * b) % m
    return j

In [5]:
def mod_inv(a, m):
    origin_m = m
    y, x = 0, 1

    if m == 1:
        return 0

    while a > 1:
        q = a // m
        temp = m

        m = a % m
        a = temp
        temp = y

        y = x - q * y
        x = temp

    if x < 0:
        x += origin_m
    return x

In [6]:
df = pd.read_csv('6_CC_Encrypted_Data.csv')
df_org = pd.read_csv('1_CC_Structured_Data.csv')
private_keys = load_key("3_CC_Private_Key.pkl")

In [7]:
column_names = [
    "Age", "Sex", "ChestPainType", "RestingBloodPressure", "Cholesterol",
    "FastingBloodSugar", "RestingECG", "MaximumHeartRate", "ExerciseAngina",
    "Oldpeak", "ST_Slope", "HeartDisease"
]

df_org = pd.read_csv("1_CC_Structured_Data.csv") 
mismatch_summary = {}  

for k in range(12):
    if k != 11:
        private_key = private_keys[k]
        x = private_key['x']
        p = private_key['p']
        g = private_key['g']

    messages = df.iloc[:, k].values
    original_column = df_org.loc[:, column_names[k]].values  

    decrypted_column = []
    mismatches = [] 

    for i, (message, original_value) in enumerate(zip(messages, original_column), 1):
        if message is not None and message == message:

            if k == 11:
                if original_value == 'Yes' and message == 1:
                    pass

                elif original_value == 'No' and message == 0:
                    pass
                
                else:
                    mismatches.append(i)
                    print(f"dec: {message}, org: {original_value}")

            else:
                c1, c2 = map(int, message.strip("()").split(","))

                cur = power(c1, x, p)
                if cur == 0:
                    decrypted_column.append(None)
                    continue
                
                temp = mod_inv(cur, p)
                decrypted_message = (c2 * temp) % p

                if k == 0:
                    temp_decrypted_message = decrypted_message
                elif k in [1, 2, 5, 6, 8, 10]: 
                    temp_decrypted_message = int_to_string(decrypted_message)
                elif k in [3, 4, 7, 9]: 
                    temp_decrypted_message = string_to_float(int_to_string(decrypted_message))
                
                decrypted_column.append(temp_decrypted_message)

                if temp_decrypted_message != original_value:
                    mismatches.append(i)
                    print(f"dec: {temp_decrypted_message}, org: {original_value}")
        
        else:
            decrypted_column.append(None)

    mismatch_summary[column_names[k]] = mismatches
    print(f"Mismatches for {column_names[k]}: {mismatches}")

print("\nMismatch Summary:")
for column, mismatches in mismatch_summary.items():
    print(f"{column}: {mismatches}")


Mismatches for Age: []
Mismatches for Sex: []
Mismatches for ChestPainType: []
Mismatches for RestingBloodPressure: []
Mismatches for Cholesterol: []
Mismatches for FastingBloodSugar: []
Mismatches for RestingECG: []
Mismatches for MaximumHeartRate: []
Mismatches for ExerciseAngina: []
Mismatches for Oldpeak: []
Mismatches for ST_Slope: []
Mismatches for HeartDisease: []

Mismatch Summary:
Age: []
Sex: []
ChestPainType: []
RestingBloodPressure: []
Cholesterol: []
FastingBloodSugar: []
RestingECG: []
MaximumHeartRate: []
ExerciseAngina: []
Oldpeak: []
ST_Slope: []
HeartDisease: []
